In [1]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.multioutput import MultiOutputRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import joblib
import pandas as pd
import numpy as np
from sklearn.metrics import mean_absolute_percentage_error
from scipy.stats import zscore

# Load and preprocess the dataset

In [8]:
df = pd.read_csv("/Users/sanghvi/Desktop/SIH/dataset/2_dec_final_short.csv")

# Define features and target

In [9]:
y = df[['UTS', 'Elongation', 'Conductivity']]
X = df[['SI', 'FE', 'MetalTemp', 'CastingWheel_RPM', 'CoolingWaterPressure',
        'CoolingWaterTemp', 'CastBarEntryTemp', 'RollingMill_RPM', 'EmulsionTemp',
         'EmulsionPressure', 'RodQuenchWaterPressure']]

# Train-test split

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize the features

In [11]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Define the model

In [12]:
rf_base = MultiOutputRegressor(RandomForestRegressor(random_state=42))

# Hyperparameter tuning using GridSearchCV

In [ ]:
param_grid = {
    "estimator__n_estimators": [100, 200, 300],
    "estimator__max_depth": [10, 20, 30],
    "estimator__min_samples_split": [2, 5, 10],
    "estimator__min_samples_leaf": [1, 2, 4]
}

grid_search = GridSearchCV(
    estimator=rf_base,
    param_grid=param_grid,
    cv=5,
    scoring='r2',
    n_jobs=-1,
    verbose=2
)

# Fit the GridSearchCV
grid_search.fit(X_train, y_train)

# Best parameters
best_rf = grid_search.best_estimator_
print("Best parameters from GridSearchCV:", grid_search.best_params_)

Fitting 5 folds for each of 81 candidates, totalling 405 fits
[CV] END estimator__max_depth=10, estimator__min_samples_leaf=1, estimator__min_samples_split=2, estimator__n_estimators=100; total time=   2.5s
[CV] END estimator__max_depth=10, estimator__min_samples_leaf=1, estimator__min_samples_split=2, estimator__n_estimators=100; total time=   2.7s
[CV] END estimator__max_depth=10, estimator__min_samples_leaf=1, estimator__min_samples_split=2, estimator__n_estimators=100; total time=   2.9s
[CV] END estimator__max_depth=10, estimator__min_samples_leaf=1, estimator__min_samples_split=2, estimator__n_estimators=100; total time=   2.7s
[CV] END estimator__max_depth=10, estimator__min_samples_leaf=1, estimator__min_samples_split=2, estimator__n_estimators=100; total time=   2.7s
[CV] END estimator__max_depth=10, estimator__min_samples_leaf=1, estimator__min_samples_split=2, estimator__n_estimators=200; total time=   5.2s
[CV] END estimator__max_depth=10, estimator__min_samples_leaf=1, est

# Evaluate the optimized model

In [14]:
y_pred = best_rf.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred)

print("\nOptimized Random Forest Performance:")
print(f"Mean Squared Error (MSE): {mse:.4f}")
print(f"Root Mean Squared Error (RMSE): {rmse:.4f}")
print(f"R^2 Score: {r2:.4f}")


Optimized Random Forest Performance:
Mean Squared Error (MSE): 1.4756
Root Mean Squared Error (RMSE): 1.2147
R^2 Score: 0.6411


# Save the Model

In [17]:
# Save the scaler and trained model as .pkl files
joblib.dump(scaler, 'random_forest_scaler.pkl')
joblib.dump(best_rf, 'optimized_random_forest_model.pkl')
print("\nScaler and model have been saved as 'random_forest_scaler.pkl' and 'optimized_random_forest_model.pkl'.")


Scaler and model have been saved as 'random_forest_scaler.pkl' and 'optimized_random_forest_model.pkl'.


# Load the saved model and scaler

In [4]:
def load_model_and_scaler():
    loaded_scaler = joblib.load('/Users/sanghvi/Desktop/SIH/Models_5/PRED/random_forest_scaler.pkl')
    loaded_model = joblib.load('/Users/sanghvi/Desktop/SIH/Models_5/PRED/optimized_random_forest_model.pkl')
    return loaded_scaler, loaded_model

# Accept runtime input and make predictions

In [5]:
def get_runtime_input():
    print("\nEnter the input parameters for prediction:")

    # Define input ranges for each parameter
    while True:
        try:
            SI = float(input("SI (%) [0.05-0.13]: "))
            if .05 <= SI <= .13:
                break
            else:
                print("Invalid input! Please enter a value between 0.05-0.13.")
        except ValueError:
            print("Invalid input! Please enter a numeric value.")

    while True:
        try:
            FE = float(input("FE (%) [0.10-0.30]: "))
            if 0.10 <= FE <= 0.30:
                break
            else:
                print("Invalid input! Please enter a value between 0.10-0.30")
        except ValueError:
            print("Invalid input! Please enter a numeric value.")

    while True:
        try:
            MetalTemp = float(input("Metal temperature (°C) [160-1100]: "))
            if 160 <= MetalTemp <= 1100:
                break
            else:
                print("Invalid input! Please enter a value between 160 and 1100.")
        except ValueError:
            print("Invalid input! Please enter a numeric value.")

    while True:
        try:
            CastingWheel_RPM = float(input("Casting Wheel RPM [1-2.5]: "))
            if 1 <= CastingWheel_RPM <= 2.5:
                break
            else:
                print("Invalid input! Please enter a value between 1-2.5")
        except ValueError:
            print("Invalid input! Please enter a numeric value.")

    while True:
        try:
            CoolingWaterPressure = float(input("Cooling Water Pressure (bar) [0-5]: "))
            if 0 <= CoolingWaterPressure <= 5:
                break
            else:
                print("Invalid input! Please enter a value between 0-5.")
        except ValueError:
            print("Invalid input! Please enter a numeric value.")

    while True:
        try:
            CoolingWaterTemp = float(input("Cooling Water Temperature (°C) [5-45]: "))
            if 5 <= CoolingWaterTemp <= 45:
                break
            else:
                print("Invalid input! Please enter a value between 5 - 45")
        except ValueError:
            print("Invalid input! Please enter a numeric value.")

    while True:
        try:
            CastBarEntryTemp = float(input("Cast Bar Entry Temprature (°C) [250-650]: "))
            if 250 <= CastBarEntryTemp <= 650:
                break
            else:
                print("Invalid input! Please enter a value between 250-650.")
        except ValueError:
            print("Invalid input! Please enter a numeric value.")

    while True:
        try:
            RollingMill_RPM = float(input("Rolling Mill RPM [400-900]: "))
            if 400 <= RollingMill_RPM <= 900:
                break
            else:
                print("Invalid input! Please enter a value between 400-900.")
        except ValueError:
            print("Invalid input! Please enter a numeric value.")

    while True:
        try:
            EmulsionTemperature = float(input("Emulsion Temperature [35-65]: "))
            if 35 <= EmulsionTemperature <= 65:
                break
            else:
                print("Invalid input! Please enter a value between 35-65.")
        except ValueError:
            print("Invalid input! Please enter a numeric value.")

    while True:
        try:
            EmulsionPressure = float(input("Emulsion Pressure [1.4-3]: "))
            if 1.4 <= EmulsionPressure <= 3:
                break
            else:
                print("Invalid input! Please enter a value between 1.4-3.")
        except ValueError:
            print("Invalid input! Please enter a numeric value.")

    while True:
        try:
            RodQuenchWaterPressure = float(input("Rod Quench Water Chamber Pressure [0-415]: "))
            if 0 <= RodQuenchWaterPressure <= 415:
                break
            else:
                print("Invalid input! Please enter a value between 0-415.")
        except ValueError:
            print("Invalid input! Please enter a numeric value.")

    # Return the inputs as a DataFrame
    user_input = pd.DataFrame({
        'SI': [SI],
        'FE': [FE],
        'MetalTemp': [MetalTemp],
        'CastingWheel_RPM': [CastingWheel_RPM],
        'CoolingWaterPressure': [CoolingWaterPressure],
        'CoolingWaterTemp': [CoolingWaterTemp],
        'CastBarEntryTemp': [CastBarEntryTemp],
        'RollingMill_RPM': [RollingMill_RPM],
        'EmulsionTemp': [EmulsionTemperature],
        'EmulsionPressure': [EmulsionPressure],
        'RodQuenchWaterPressure': [RodQuenchWaterPressure]
    })

    return user_input

# Function to predict properties based on user input

In [6]:
def make_prediction_from_user_input():
    
    # Load scaler and model
    scaler, model = load_model_and_scaler()
    
    # Get user input
    user_input = get_runtime_input()
    
    # Scale the user input
    user_input_scaled = scaler.transform(user_input)
    
    # Make predictions
    predictions = model.predict(user_input_scaled)
    
    # Display the predictions
    print("\nPredicted Properties:")
    print(f"UTS (Ultimate Tensile Strength): {predictions[0][0]:.2f}")
    print(f"Elongation: {predictions[0][1]:.2f}")
    print(f"Conductivity: {predictions[0][2]:.2f}")

# Main loop to allow repeated predictions

In [7]:
while True:
    print("\n--- Aluminum Wire Rod Property Prediction ---")
    make_prediction_from_user_input()
    cont = input("\nDo you want to make another prediction? (yes/no): ").strip().lower()
    if cont != 'yes':
        print("Exiting prediction tool. Goodbye!")
        break


--- Aluminum Wire Rod Property Prediction ---

Enter the input parameters for prediction:

Predicted Properties:
UTS (Ultimate Tensile Strength): 11.48
Elongation: 8.65
Conductivity: 61.82
Exiting prediction tool. Goodbye!
